In [1]:
import APIs
import data_mongo as dm 


In [2]:
# exchange = APIs.currency_api('USD')
# exchange['rates']

exchange = {'rates':{
                    'USD': 1,
                    'EUR': 0.906638,
                    'JPY': 108.679651,
                    'GBP': 0.776193,
                    'CAD': 1.323993,
                    'SEK': 9.667363
                    }
           }
exchange['rates']['EUR']

0.906638

In [3]:
# Conexion

comps_db, comps_coll = dm.connect_collection('companies', 'companies')

In [4]:
# Borrar las que hayan quebrado

comps_coll.delete_many({'deadpooled_year':{'$gte':0}})

In [5]:
# Borrar las que no tengan oficina

comps_coll.delete_many({'offices':{'$eq':[]}})

In [6]:
#Hacer lista para actualizar 

comps_dollar = list(comps_coll.find({'deadpooled_year': None}))

In [7]:
#Actualización de los datos para ver la cantidad en dolares

for comp in comps_dollar:
    value_dollar = {"$set": {'raised_dollar':dm.get_raised_dollar(comp, exchange)}}
    comps_coll.update_one(comp, value_dollar)

In [8]:
# Por cada oficina, crear un nuevo campo

unpack = [{'$unwind':'$offices'},{'$project': {'_id':0}}, {'$out':'companies'}]
comps_coll.aggregate(unpack)

In [9]:
# Borrar aquellas sin coordenadas

comps_coll.delete_many({'$or':[{"offices.latitude":{'$eq':None}},{"offices.longitude":{'$eq':None}}]})

In [10]:
# Borrar gilipolleces

comps_coll.update_many({}, {'$unset': {'permalink':1, 'crunchbase_url':1, 'homepage_url':1, 'blog_url':1,
                                   'blog_feed_url':1, 'twitter_username':1,'founded_month':1, 'founded_day':1,
                                   'deadpooled_year':1,'deadpooled_month':1, 'deadpooled_day':1, 'deadpooled_url':1,
                                   'tag_list':1, 'alias_list':1, 'email_address':1, 'phone_number':1, 'created_at':1,
                                   'updated_at':1, 'overview':1, 'image':1, 'products':1, 'relationships':1, 'competitions':1,
                                   'providerships':1, 'milestones':1, 'ipo':1, 'video_embeds':1,'screenshots':1,'external_links':1,
                                   'partners':1, 'acquisition':1, 'acquisitions':1, 'investments': 1 }})

In [11]:
#lista para las localizaciones

comps_loc = list(comps_coll.find())

In [12]:
#poner las localizaciones en el formato requerido

for comp in comps_loc:
    value_loc = {'$set': {'location': dm.get_location(comp)}}
    comps_coll.update_one(comp, value_loc)